Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
from bs4 import BeautifulSoup
print('installed')

installed


In [2]:
html_file='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_file=requests.get(html_file) #get website html
soup= BeautifulSoup(html_file.text, 'lxml') #.text turns it to readable format
#soup.prettify makes indents
ws=soup.find('table', class_='wikitable sortable') #localize to extract the table contents
# print(ws) 
# show you the whole contents, which gives the "tree branches" specific to that webpage

In [3]:
for full in soup.find_all('table', class_='wikitable sortable'):
    
    tablecontent=full.tbody.text
    #print(tablecontent)
    #learning how to access html contents

In [4]:
tablecontent=tablecontent.split('\n') #turn results into a separated list
#print(tablecontent)

In [5]:
while '' in tablecontent: tablecontent.remove('')
#print(tablecontent)

In [6]:
import pandas as pd
data=list(zip(*[iter(tablecontent)]*3)) #sort list into 3 columns
df=pd.DataFrame(data[1:],columns=data[0])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
df.columns #check to see only 3 columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [8]:
import numpy as np

# replace "Not assigned" to NaN
df.replace("Not assigned", np.nan, inplace = True)
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
df.dropna(thresh=2, inplace=True) 
#drop all rows who both have NaN because thresh 2 means only rows w 2 columns that have NaN get dropped
#this keeps Boroughs w/o neighborhood names in the dataframe
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,NaN
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [10]:
df['Neighbourhood'].replace(np.nan, df['Borough'], inplace=True)
#replace all NaN with the name of the Borough from the same row
df.reset_index(drop=True, inplace=True)
#reorder the index because rows were dropped
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


need to now combine postcodes into single lines.
making assumption that all boroughs are same when the postcode is the same.

In [13]:
combo= df.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(lambda col: ', '.join(col))
combo.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [12]:
combo.shape

(103, 1)